### 1. 최근접 이웃 협업 필터링


##### 협업 필터링 방식 중 하나인 최근접 이웃 협업 필터링에는 사용자 기반 최근접 이웃 방식과 아이템 기반 최근접 이웃 방식 두가지의 방법이 있습니다.

```

1-1) 사용자 기반 최근접 이웃 방식과 아이템 기반 최근접 이웃 방식에 대해 간단히 설명하시요.

```

```
답: 사용자 기반 최근접 이웃 방식은 사용자와 아이템 선호도가 비슷한 고객들이 구매한 아이템을 추천하는 방식입니다.
   특정 사용자와 타 사용자 간의 유사도를 측정한 뒤 가장 유사도가 높은 사용자를 추출해 그들이 선호하는 아이템을 추천하는 방식입니다.
   
   반면 아이템 기반 최근접 이웃 방식은 이 아이템을 구매한 고객들이 구매한 다른 아이템을 추천하는 방식입니다.
   사용자가 평가한 아이템의 평가가 유사하게 나타나는 아이템을 추천하는 방식입니다.
```

```
1-2) 사용자 기반 최근접 이웃 방식보다 아이템 기반 최근접 이웃 방식이 더 정확도가 높고 많이 사용하는 이유를 설명해 보시오.
```

```
답: 비슷한 아이템을 좋아한다고 해서 사람들의 취향이 비슷하다고 판단하기에는 어려운 경우가 있습니다. 예를 들어 매우 유명한 아이템의 경우 취향과는 상관없이 구매하는 경향이 있고, 사용자가 평점을 준 아이템의 개수가 많지 않을 때 사용자 간 비교가 용이하지 않습니다. 따라서 사용자 기반 최근접 이웃 방식보다 아이템 기반 최근접 이웃방식을 더 많이 사용합니다.
```

### 2. 행렬분해

##### 2-1) 다음 행렬에 대해서 경사하강법을 이용한 행렬분해를 실습한 후 원본행렬과 비교해 보세요.

* 조건
```
K = 3
steps = 1000
learning_rate = 0.01
r_lambda = 0.01
```

In [1]:
import numpy as np

In [2]:
# 5X6 원본행렬

R = np.array([[1, 5, np.NaN, 2, np.NaN, np.NaN],
              [3, np.NaN, 1, 4, 3, np.NaN],
              [5, np.NaN, 2, 3, np.NaN, 1],
              [2, 5, np.NaN, 1, 2, 2],
              [3, 4, 2, np.NaN, 1, 1]
    
])

np.random.seed(1)

In [3]:
num_users, num_items = R.shape

K = 3

P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [7]:
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

In [8]:
for step in range(steps):
    for i, j, r in non_zeros:
        eij = r - np.dot(P[i, :], Q[j, :].T)
        
        P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])

In [9]:
pred_matrix = np.dot(P, Q.T)
print('예측행렬: \n', np.round(pred_matrix, 3))

예측행렬: 
 [[1.019 4.978 1.029 1.991 2.82  2.554]
 [2.996 5.44  0.999 3.969 2.983 2.393]
 [4.961 4.678 2.004 2.992 1.31  0.995]
 [1.983 4.991 1.881 1.025 1.985 1.967]
 [2.997 3.974 1.973 0.991 1.012 1.022]]


In [7]:
print(R)

[[ 1.  5. nan  2. nan nan]
 [ 3. nan  1.  4.  3. nan]
 [ 5. nan  2.  3. nan  1.]
 [ 2.  5. nan  1.  2.  2.]
 [ 3.  4.  2. nan  1.  1.]]


##### 2-2) 다음 RMSE를 계산하는 함수를 사용하여 오차를 확인하세요.

In [8]:
# RMSE
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [9]:
rmse = get_rmse(R, P, Q, non_zeros)
print(rmse)

0.02026146779611784


### 3. 콘텐츠 기반 필터링

##### Book.csv 파일을 로드하고 콘텐츠 기반 필터링 방법을 사용하여 ㅇㅇ와 관련있는 10개의 책을 추천하는 실습입니다.

In [10]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

##### 3-1) Book.csv 파일을 로드하고 결측값이 있는 행을 제거하고 Genres 행렬을 단어의 나열로 전처리 한 후 Countvectorizer을 사용하여 백터행렬을 생성하시오.
* 조건</br>
CountVectorizer(min_df=0, ngram_range=(1,2))

In [11]:
data = pd.read_csv('/Users/yeins/YEIN/비타민/Book reviews/Book.csv')
data.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Publisher,genres
0,276725,3,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,276726,5,Rites of Passage,Judith Rae,Heinle,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,276727,4,The Notebook,Nicholas Sparks,Warner Books,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,276729,3,Help!: Level 1,Philip Prowse,Cambridge University Press,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,Cambridge University Press,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      500 non-null    int64 
 1   Book-Rating  500 non-null    int64 
 2   Book-Title   370 non-null    object
 3   Book-Author  370 non-null    object
 4   Publisher    370 non-null    object
 5   genres       500 non-null    object
dtypes: int64(2), object(4)
memory usage: 23.6+ KB


In [13]:
data = data.dropna()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370 entries, 0 to 499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      370 non-null    int64 
 1   Book-Rating  370 non-null    int64 
 2   Book-Title   370 non-null    object
 3   Book-Author  370 non-null    object
 4   Publisher    370 non-null    object
 5   genres       370 non-null    object
dtypes: int64(2), object(4)
memory usage: 20.2+ KB


In [15]:
pd.set_option('max_colwidth', 100)
data['genres'][:1]

0    [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {...
Name: genres, dtype: object

In [16]:
from ast import literal_eval

data['genres'] = data['genres'].apply(literal_eval)

In [17]:
data['genres'] = data['genres'].apply(lambda x : [ y['name'] for y in x])
data.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Publisher,genres
0,276725,3,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"[Action, Adventure, Fantasy, Science Fiction]"
1,276726,5,Rites of Passage,Judith Rae,Heinle,"[Adventure, Fantasy, Action]"
2,276727,4,The Notebook,Nicholas Sparks,Warner Books,"[Action, Adventure, Crime]"
3,276729,3,Help!: Level 1,Philip Prowse,Cambridge University Press,"[Action, Crime, Drama, Thriller]"
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge English Readers),Sue Leather,Cambridge University Press,"[Action, Adventure, Science Fiction]"


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

data['genres_literal'] = data['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(data['genres_literal'])
print(genre_mat.shape)

(370, 147)


##### 3-2) 생성된 백터 행렬으로 코사인 유사도를 계산하고 행렬로 변환한 뒤 유사도가 높은 순으로 정렬하시오.

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)

(370, 370)


In [20]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

##### 3-3) 장르 유사도에 따라 영화를 추천하는 다음 함수의 '빈칸'을 채우고 'The Notebook'이라는 책을 좋아하는 사용자에게 보여줄 추천 결과를 출력하시오(10개).

In [21]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    title_movie = df[df['Book-Title'] == title_name]
    
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [22]:
similar_movies = find_sim_movie(data, genre_sim_sorted_ind, 'The Notebook',10)
similar_movies[['Book-Title', 'Book-Rating', 'Book-Author']]

[[  2 268 351  18 237  46 297  17  24  32]]


,Book-Title,Book-Rating,Book-Author
2,The Notebook,4,Nicholas Sparks
356,Oceano Mare,6,Alessandro Baricco
469,Contacto,6,Carl Sagan
21,How to Deal With Difficult People,7,Rick Brinkman
304,Verraten und verkauft. Roman.,3,Philippe Djian
62,Henry der Held.,10,Roddy Doyle
411,The Tower at Stony Wood,8,Patricia A. McKillip
20,Birdsong: A Novel of Love and War,8,Sebastian Faulks
29,The Pillars of the Earth,5,Ken Follett
38,Besiegt vom Sturm der Leidenschaft.,4,Patricia Potter
